# Reverse-engineering of the surron ultrabee can bus

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

In [49]:

with open('20240427_CAN_Trace_Motorrad_T000.csv', 'r') as f:
    reader = csv.reader(f, delimiter=";")
    data = list(reader)

data.pop(0)

df = pd.DataFrame(data)
df

,0,1,2,3,4
0,0.008923,Rx,8,00 00 00 00 02 00 00 02,18
1,0.017042,Rx,8,25 03 00 00 00 00 00 28,258
2,0.017292,Rx,8,00 00 00 00 00 00 00 00,259
3,0.017540,Rx,8,B2 20 00 00 00 00 00 D2,260
4,0.017788,Rx,8,10 27 00 00 00 00 00 37,261
...,...,...,...,...,...
38845,106.559.131,Rx,8,FE FF 2C 04 5A 00 00 87,259
38846,106.559.379,Rx,8,B2 20 00 00 00 00 00 D2,260
38847,106.559.627,Rx,8,10 27 00 00 00 00 00 37,261
38848,106.559.916,Rx,8,00 03 E7 03 28 00 C2 0F,417333248x
